# Utilities 2: Quantum Clock Analysis

## Overview

**Fractal peel quantum clock analysis** using Riemann zeta zero spacings.

### What is a Quantum Clock?

Uses **spacings between consecutive zeta zeros** as a quantum timing reference with:
- Deterministic mathematical definition
- Fractal multi-scale structure
- Quantum-like discrete ticks
- Universal reference frame

### Key Techniques

**Fractal Peel**: Haar wavelet variance cascade
- Decomposes signal across scales
- Measures fractal dimension, coherence time, residual fraction

**Spectral Analysis**: Power spectrum analysis
- Identifies dominant frequencies
- Measures spectral sharpness

**Architecture**: Layer 2 Core (`workbench.core.quantum`)

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Layer 2: Core primitives
from workbench.core.quantum import QuantumClock
from workbench.core.zeta import ZetaFiducials

print('✓ Imports successful')

## 1. Creating a Quantum Clock

Initialize with specified number of zeta zeros.

In [ ]:
# Create quantum clock with 200 zeros
qc = QuantumClock(n_zeros=200)
print(f'✓ Quantum Clock initialized with {qc.n_zeros} zeros')

## 2. Computing Zeta Zero Spacings

Spacings between consecutive zeros form the clock signal.

In [ ]:
spacings = qc.compute_zeta_spacings()
print(f'Computed {len(spacings)} spacings')
print(f'Mean: {np.mean(spacings):.6f}, Std: {np.std(spacings):.6f}')

# Visualize
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(spacings, 'o-', markersize=3, alpha=0.6)
plt.title('Zeta Zero Spacings')
plt.subplot(1, 2, 2)
plt.hist(spacings, bins=30, alpha=0.7, edgecolor='black')
plt.title('Spacing Distribution')
plt.tight_layout()
plt.show()

## 3. Fractal Peel Analysis

Haar wavelet variance cascade reveals multi-scale structure.

**Metrics**:
- Fractal Dimension: 0.5=white noise, <0.5=anti-persistent, >0.5=persistent
- Coherence Time: Scale where variance drops to 50%
- Residual Fraction: Final/initial variance ratio

In [ ]:
fractal_data = qc.fractal_peel(spacings, max_levels=7)
print(f'Fractal Dimension: {fractal_data["fractal_dim"]:.3f}')
print(f'Coherence Time: {fractal_data["coherence_time"]} ticks')
print(f'Residual Fraction: {fractal_data["resfrac"]:.2e}')

# Visualize variance cascade
plt.figure(figsize=(10, 5))
plt.semilogy(range(len(fractal_data['variances'])), fractal_data['variances'], 'o-')
plt.xlabel('Level')
plt.ylabel('Variance')
plt.title('Fractal Peel: Variance Cascade')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Spectral Analysis

Power spectrum reveals frequency-domain structure.

In [ ]:
freqs, power = qc.compute_structure_factor(spacings)
sharpness = qc.compute_spectral_sharpness(freqs, power)
peak_idx = np.argmax(power)
dominant_freq = freqs[peak_idx]

print(f'Spectral Sharpness: {sharpness:.3f}')
print(f'Dominant Frequency: {dominant_freq:.6f}')

plt.figure(figsize=(10, 5))
plt.semilogy(freqs, power)
plt.axvline(dominant_freq, color='r', linestyle='--', label=f'Peak: {dominant_freq:.4f}')
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.title('Power Spectrum')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Complete Analysis

Run full suite of metrics in one call.

In [ ]:
qc2 = QuantumClock(n_zeros=300)
metrics = qc2.analyze()

print('Complete Metrics:')
for key, value in sorted(metrics.items()):
    if isinstance(value, float):
        print(f'  {key:25s}: {value:.6f}')
    else:
        print(f'  {key:25s}: {value}')

## 6. Riemann Hypothesis Falsification Test

Tests sensitivity to zeros off critical line.

**Interpretation**:
- resfrac < 0.05: Consistent with RH
- resfrac > 0.05: Would falsify RH

In [ ]:
# Test RH case vs shifted case
resfrac_rh, falsify_rh = qc2.test_rh_falsification(beta_shift=0.0, n_test=100)
resfrac_shift, falsify_shift = qc2.test_rh_falsification(beta_shift=0.01, n_test=100)

print(f'RH case (β=0):      resfrac={resfrac_rh:.4f}, Falsifies: {falsify_rh}')
print(f'Shifted (β=0.01):   resfrac={resfrac_shift:.4f}, Falsifies: {falsify_shift}')

# Visualize
plt.figure(figsize=(8, 5))
plt.bar(['RH Case', 'Shifted'], [resfrac_rh, resfrac_shift], 
        color=['green', 'red'], alpha=0.7)
plt.axhline(0.05, color='orange', linestyle='--', label='Threshold')
plt.ylabel('Residual Fraction')
plt.title('RH Falsification Test')
plt.legend()
plt.show()

## 7. Scaling Analysis

How metrics scale with number of zeros.

In [ ]:
print(f'{"N Zeros":<10} {"Time (s)":<12} {"Fractal Dim":<15}')
for n in [50, 100, 200, 400]:
    start = time.time()
    qc_test = QuantumClock(n_zeros=n)
    m = qc_test.analyze()
    elapsed = time.time() - start
    print(f'{n:<10} {elapsed:<12.2f} {m["fractal_dim"]:<15.3f}')

## 8. Custom Signal Analysis

Apply methods to any signal for fractal/spectral analysis.

In [ ]:
# Analyze custom signal
np.random.seed(42)
t = np.linspace(0, 10, 1000)
signal = np.sin(2*np.pi*t) + 0.5*np.sin(4*np.pi*t) + 0.2*np.random.randn(len(t))

qc_custom = QuantumClock(n_zeros=100)
fractal_custom = qc_custom.fractal_peel(signal, max_levels=8)

print(f'Custom Signal Fractal Dimension: {fractal_custom["fractal_dim"]:.3f}')
print(f'Custom Signal Coherence Time: {fractal_custom["coherence_time"]} samples')

# Compare with white noise
noise = np.random.randn(1000)
fractal_noise = qc_custom.fractal_peel(noise, max_levels=8)
print(f'White Noise Fractal Dimension: {fractal_noise["fractal_dim"]:.3f}')

## Summary

### Key Takeaways
1. Quantum clock uses zeta spacings as timing reference
2. Fractal peel reveals multi-scale structure
3. Spectral analysis identifies dominant frequencies
4. RH test shows sensitivity to off-line zeros
5. Methods apply to any signal

### Applications
- Precision timing
- Signal processing
- Pattern detection
- RH verification

### Next Steps
- **Utilities 1**: Fast zeta computation
- **Techniques 1**: See how used in processors

**Architecture**: `workbench/core/quantum.py` (Layer 2)